In [1]:
###!pip install -q langchain_community langchain langchain_openai

In [2]:
import langchain

langchain.__version__

'0.0.326'

In [3]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://wikidocs.net/231393'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

1
16070



                        
                        2-3-2. RecursiveCharacterTextSplitter
                        





                        
                        2-3-3. 토큰 수를 기준으로 텍스트 분할 (Tokenizer 활용)
                        





                        
                        2-4. RAG - Embedding
                        





                        
                        2-4-1. OpenAIEmbeddings
                        





                        
                        2-4-2. HuggingFaceEmbeddings
                        





                        
                        2-4-3. GoogleGenerativeAIEmbeddings
                        





                        
                        2-5. RAG - Vector Store
                        





                        
                        2-5-1. Chroma
                        





                        
                        2-5-1-1. 유사도 기반 검색 (Similarity search)
                        





             

In [4]:
# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter
#chunk_size: 각 조각의 크기는 1000글자, chunk_overlap: 오버랩은 200자
#RecursiveCharacterTextSplitter을 사용했기 때문에 문장이 자연스럽게 마무리 됨!
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[0])

20
page_content='2-1. RAG 개요 - 랭체인(LangChain) 입문부터 응용까지\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTOC\n\n Show\n                \n\n Hide\n                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n랭체인(LangChain) 입문부터 응용까지\n\n\n\n\n\n                        \n                        Part 0. 글쓴이 소개\n                        \n\n\n\n\n\n                        \n                        Part 1. LangChain 기초\n                        \n\n\n\n\n\n                        \n                        1-1. LangChain 이란?\n                        \n\n\n\n\n\n                        \n                        1-1-1. LangChain v0.1.0 출시 의미\n                        \n\n\n\n\n\n                        \n                        1-1-2. LangChain 프레임워크의 구성\n                        \n\n\n\n\n\n                        \n                        1-1-3. 필수 라이브러리 설치\n                        \n\n\n\n\n\n                        \n                        1-1-4. OpenAI 인증키  등록\n                        \n\n\n\n

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os

os.environ['OPENAI_API_KEY'] = input("OPAI-key")

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

In [6]:
import transformers
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# HuggingFace 모델 설정
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
MY_HF_TOKEN = input("hugging-key")

# 파이프라인 설정
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
    token=MY_HF_TOKEN
)

# RAG에 사용할 Prompt 템플릿 정의
template = '''Answer the question based only on the following context:
{context}

Question: {question}
Answer:'''
prompt = ChatPromptTemplate.from_template(template)

# 검색된 문서들을 문자열로 변환하는 함수 정의
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs if hasattr(doc, 'page_content'))

# RAG Chain 정의
retriever = vectorstore.as_retriever()  # 벡터스토어에서 검색기 생성

# HuggingFace 파이프라인을 사용하는 함수
def run_llm(prompt_value):
    # ChatPromptValue를 문자열로 변환
    prompt_str = prompt_value.to_string()
    response = pipeline(
        prompt_str,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    return response[0]['generated_text']

# RAG Chain 연결
rag_chain = (
    {
        'context': retriever | format_docs,
        'question': RunnablePassthrough()
    }
    | prompt
    | run_llm
    | StrOutputParser()
)

# 질문 실행
question = "RAG에 대해 설명해주세요."
result = rag_chain.invoke(question)

# 결과 출력
print(result)


/data/sllm/src/finetuning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.26s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Human: Answer the question based only on the following context:
2-1. RAG 개요
                        





                        
                        2-2. RAG - Document Loader
                        





                        
                        2-2-1. 웹 문서 (WebBaseLoader)
                        





                        
                        2-2-2. 텍스트 문서 (TextLoader)
                        





                        
                        2-2-3. 디렉토리 폴더 (DirectoryLoader)
                        





                        
                        2-2-4. CSV 문서 (CSVLoader)
                        





                        
                        2-2-5. PDF 문서 
                        





                        
                        2-2-5-1. PDF 문서 페이지별로 로드 (PyPDFLoader)
                        





                        
                        2-2-5-2. 형식이 없는 PDF 문서 로드  (UnstructuredPDFLoader)

4-2. 사용자 정의 (custom) 도구
                     

In [7]:
# 질문 실행
question = "내용 요약해줘."
result = rag_chain.invoke(question)

# 결과 출력
print(f"result: {result}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result: Human: Answer the question based only on the following context:
2-1. RAG 개요 - 랭체인(LangChain) 입문부터 응용까지


























TOC

 Show
                

 Hide
                



















랭체인(LangChain) 입문부터 응용까지





                        
                        Part 0. 글쓴이 소개
                        





                        
                        Part 1. LangChain 기초
                        





                        
                        1-1. LangChain 이란?
                        





                        
                        1-1-1. LangChain v0.1.0 출시 의미
                        





                        
                        1-1-2. LangChain 프레임워크의 구성
                        





                        
                        1-1-3. 필수 라이브러리 설치
                        





                        
                        1-1-4. OpenAI 인증키  등록
                        





                        
                        1-2. LL